In [5]:
## In this notebook, we will attempt to find the magnitude of nongravitational 
## acceleration for which heliolinc and link_purify will no longer recognize 
## a given object as a single body. 

In [1]:
## first, import the necessary modules

import os 
import sys
import numpy as np
import pandas as pd
import subprocess
import sorcha

In [44]:
## create the Sorcha input files for one single object 
## the object could be a normal object like Pratchett, 
## or one that already displays nongravs. Favor the former.

## note that there are two ways to approach this overall:

## 1 - Run the ENTIRE PIPELINE for each object with each nongrav setting,
##   - creating different input files each time

## 2 - Figure out a way to determine what object the linkages in a given 
##   - heliolinx output correspond to, whether using orbital element
##   - comparison or some other way. 

## in any case, I believe that the best course of action is to create
## separate input files for each magnitude of nongrav -- i.e., one input 
## file where the A1 magnitude is 10^-6, one input file where the A1 magnitude
## is 10^-5, etc., for each object in the file. 

In [2]:
## define our filepaths and filename stems

## filepath for sorcha input and output files
sfpath = "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/"

## filename stems for the orbit files and other files
fname_orb = "pratchett_orb"
fname_stem = "pratchett"

## object ID
obj_id = "127005Pratchett"

## location of Sorcha config file
config_fpath = "Rubin_full_footprint.ini"

## location of pointing database file for Sorcha
pointing_db_path = "/home/ellie/src/sorcha/sorcha/demo/baseline_v2.0_1yr.db"

## location of Earth location file for HelioLinC
earth1day_path = '/home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv'

## location of ObsCodes file for HelioLinC
obscodes_path = '/home/ellie/research/lsst/heliolinc_files/ObsCodes.html'

## location of colformat file for HelioLinC
colformat_path = '/home/ellie/research/lsst/heliolinc_files/colformat.txt'

## location of hypothesis file for HelioLinC
hypo_path = '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/hihyp00b_mb.txt'

## location of kep2cart.py program
k2c_path = '/home/ellie/research/lsst/lsst_seti/nongrav/sorcha_ng'

## import the kep2cart module
kep_dir = os.path.abspath(k2c_path)
sys.path.insert(0, kep_dir)

from kep2cart import kep2cart as k2c

In [3]:
## create the Sorcha inputs

## first, the orbits file(s)
## read in the Keplerian orbits file first and convert to Cartesian

k2c("{0}_kep.csv".format(fname_orb), "{0}{1}_cart.csv".format(sfpath, fname_orb))

New Cartesian orbits file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv written.


In [4]:
## generate the nongravitational orbits files
nongravs = np.logspace(start=-12, stop=3, num=16, base=10) 

df = pd.read_csv("{0}{1}_cart.csv".format(sfpath, fname_orb))
df['FORMAT'] = 'NONGRAV'
df['a1'] = 0.0
df['a2'] = 0.0
df['a3'] = 0.0
df['model'] = 'ASTEROID'
epoch = df.pop('epochMJD_TDB')
df.insert(12, 'epochMJD_TDB', epoch)

orb_files = ["{0}{1}_cart.csv".format(sfpath, fname_orb)]

for i in range(len(nongravs)):
    df1 = df.copy()
    df1['a1'] = nongravs[i]
    df1_fname = "{0}{1}_ng_a1_{2}.csv".format(sfpath, fname_orb, np.log10(nongravs[i]))
    df1.to_csv(df1_fname, index=False)

    df2 = df.copy()
    df2['a2'] = nongravs[i]
    df2_fname = "{0}{1}_ng_a2_{2}.csv".format(sfpath, fname_orb, np.log10(nongravs[i]))
    df2.to_csv(df2_fname, index=False)

    df3 = df.copy()
    df3['a3'] = nongravs[i]
    df3_fname = "{0}{1}_ng_a3_{2}.csv".format(sfpath, fname_orb, np.log10(nongravs[i]))
    df3.to_csv(df3_fname, index=False)

    orb_files.append(df1_fname)
    orb_files.append(df2_fname)
    orb_files.append(df3_fname)  

df_orbfiles = pd.DataFrame()
df_orbfiles['orb_files'] = orb_files

In [5]:
## generate Physical Parameters file using placeholder values
## (these are not the correct values, need to fix)

df_phy = pd.DataFrame()
df_phy['ObjID'] = [obj_id]
df_phy['H_r'] = [5.63]
df_phy['u-r'] = [1.508064] #[2.55]
df_phy['g-r'] = [0.293132] #[0.92]
df_phy['i-r'] = [-0.292761] #[-0.38]
df_phy['z-r'] = [-0.298066] #[-0.59]
df_phy['y-r'] = [0.302652] #[-0.70]
df_phy['GS'] = [0.15]

df_phy.to_csv("{0}{1}_phy.csv".format(sfpath, fname_stem), index=False)

In [22]:
## create a list of output filename stems for Sorcha

ofnames = []

for idx in range(len(orb_files)):

    ## define the output filename stem (no extension)
    ofname = orb_files[idx].replace("_orb", "") 
    ofname = ofname.split('/')[-1]
    ofname = ofname.split('.')[:-1]
    ofname = '.'.join(ofname)

    ofnames.append(ofname)

print(orb_files[0])
print(ofnames[0])

df_orbfiles['of_names'] = ofnames
df_orbfiles.to_csv('orb_files2.csv', index=False)

/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv
pratchett_cart


In [10]:
## run Sorcha on each file
lst = range(len(orb_files))
#print(lst[28:][0])

for i in lst:#range(len(orb_files)):
    print(orb_files[i])
    print(ofnames[i])
    subprocess.run(["sorcha","run","-c", config_fpath,"-p","{0}_phy.csv".format(fname_stem),"--orbits",\
                    orb_files[i],"--pd",pointing_db_path,"-o",sfpath,"-t",sfpath+ofnames[i],\
                    "--st","{0}{1}_stats".format(sfpath,ofnames[i])])

    print('\n{0} out of {1} ({2} %) complete'.format(i+1, len(orb_files), 100*round((i+1)/len(orb_files), 2)))

/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv
pratchett_cart

1 out of 49 (2.0 %) complete
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_ng_a1_-12.0.csv
pratchett_ng_a1_-12.0
1.0 1.0 2.0 5.093 0.0

2 out of 49 (4.0 %) complete
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_ng_a2_-12.0.csv
pratchett_ng_a2_-12.0
1.0 1.0 2.0 5.093 0.0

3 out of 49 (6.0 %) complete
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_ng_a3_-12.0.csv
pratchett_ng_a3_-12.0
1.0 1.0 2.0 5.093 0.0

4 out of 49 (8.0 %) complete
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_ng_a1_-11.0.csv
pratchett_ng_a1_-11.0
1.0 1.0 2.0 5.093 0.0

5 out of 49 (10.0 %) complete
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_ng_a2_-11.0.csv
pratchett_ng_a2_-11.0
1.0 1.0 2.0 5.093 0.0

6 out of 49 (12.0 %) complete
/home/ellie/research/lsst/sorcha_output/127005prat

In [23]:
## shorten the Sorcha output to one month of detections, as heliolinc seems to balk
## at anything much longer, according to Aren

arr = lst
out_fnames = []
missing = []
new_ofnames = []

for j in arr:

    if os.path.isfile("{0}{1}.csv".format(sfpath, ofnames[j])):
        ## insert logic to check if file exists -- if so, do the following
        ## (else, print "{fname} does not exist, skipping")
        df_sorcha = pd.read_csv("{0}{1}.csv".format(sfpath, ofnames[j]))
        start_date = df_sorcha['fieldMJD_TAI'][0]
        end_date = start_date+30

        df_sorcha_30d = df_sorcha[df_sorcha['fieldMJD_TAI'] <= end_date]
        df_sorcha_30d.to_csv("{0}{1}_30days.csv".format(sfpath, ofnames[j]), index=False)

        ## create an input file from the Sorcha
        ## output that is compatible with make_tracklets
        ## ( this same code can be found in 
        ## /lsst_seti/nongrav/heliolinc/create_maketracklets_input.py )
    
        fname = "{0}{1}_30days.csv".format(sfpath, ofnames[j])
        out_fname = "{0}{1}_30days_cmi.csv".format(sfpath, ofnames[j])

        df = pd.read_csv(fname)
        nrows = len(df['ObjID'])
        df['ObsCode'] = np.full((nrows, 1), 'I11')

        df.to_csv(out_fname, index=False)

        print(out_fname)
        out_fnames.append(out_fname)
        new_ofnames.append(ofnames[j])
        
    else:
        print("Warning: {0}{1}.csv does not exist.".format(sfpath, ofnames[j]))
        missing.append("{0}{1}.csv".format(sfpath, ofnames[j]))

df_missing = pd.DataFrame()
df_missing['missing_files'] = missing
df_missing.to_csv('missing_files.csv', index=False)

## reassign ofnames so it only includes non-missing files,
## to avoid issues later
ofnames = new_ofnames

/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a1_-12.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a2_-12.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a3_-12.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a1_-11.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a2_-11.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a3_-11.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a1_-10.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a2_-10.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_ng_a3_-10.0_30days_cmi.csv
/home/ellie/research/lsst/sorcha_outp

In [24]:
## run make_tracklets

for k in range(len(out_fnames)):

    subprocess.run(['make_tracklets', '-dets', out_fnames[k], '-earth', earth1day_path, \
                   '-obscode', obscodes_path, '-colformat', colformat_path,'-outimgs', \
                   '{0}outim_{1}.txt'.format(sfpath, ofnames[k]), \
                   '-pairdets', '{0}pairdets_{1}.csv'.format(sfpath, ofnames[k]),\
                   '-tracklets', '{0}tracklets_{1}.csv'.format(sfpath, ofnames[k]), \
                   '-trk2det', '{0}trk2det_{1}.csv'.format(sfpath, ofnames[k])])

Checking out argv[1] = -dets.
Checking out argv[3] = -earth.
Checking out argv[5] = -obscode.
Checking out argv[7] = -colformat.
Checking out argv[9] = -outimgs.
Checking out argv[11] = -pairdets.
Checking out argv[13] = -tracklets.
Checking out argv[15] = -trk2det.

Input detection file is called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_30days_cmi.csv

No input image file specified: image catalog will be generated internally.
column formatting file = /home/ellie/research/lsst/heliolinc_files/colformat.txt
Observatory code file /home/ellie/research/lsst/heliolinc_files/ObsCodes.html
Output image file will be called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt
Output paired detection file will be called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv
Output tracklet file will be called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv
O

In [25]:
## run heliolinx

for l in range(len(ofnames)):

    subprocess.run(['heliolinc', '-imgs', '{0}outim_{1}.txt'.format(sfpath, ofnames[l]), \
                    '-pairdets', '{0}pairdets_{1}.csv'.format(sfpath, ofnames[l]), \
                    '-tracklets', '{0}tracklets_{1}.csv'.format(sfpath, ofnames[l]), \
                    '-trk2det', '{0}trk2det_{1}.csv'.format(sfpath, ofnames[l]), \
                    '-obspos', earth1day_path, '-heliodist', hypo_path, '-outsum',\
                    '{0}sum_{1}.csv'.format(sfpath, ofnames[l]), '-clust2det', \
                    '{0}clust2det_{1}.csv'.format(sfpath, ofnames[l])])

Checking out argv[1] = -imgs.
Checking out argv[3] = -pairdets.
Checking out argv[5] = -tracklets.
Checking out argv[7] = -trk2det.
Checking out argv[9] = -obspos.
Checking out argv[11] = -heliodist.
Checking out argv[13] = -outsum.
Checking out argv[15] = -clust2det.
input image file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt
input detection file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv
input tracklet file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv
input trk2det file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_pratchett_cart.csv
input observer position file /home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv
input heliocentric hypothesis file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/hihyp00b_mb.txt
input reference MJD 0
Defaulting to cluster radius = 100000km
Defaulting to min. geocentric distance

In [26]:
## run link_purify -- first, create the lflist file

for m in range(len(ofnames)):
    with open('{0}{1}_lflist'.format(sfpath, ofnames[m]), 'w') as f:
        f.write("{0}sum_{1}.csv {0}clust2det_{1}.csv".format(sfpath, ofnames[m]))
        f.close()

        ## now, actually run link_purify
        subprocess.run(['link_purify','-imgs','{0}outim_{1}.txt'.format(sfpath, ofnames[m]),\
                        '-pairdets','{0}pairdets_{1}.csv'.format(sfpath, ofnames[m]), \
                        '-lflist','{0}{1}_lflist'.format(sfpath, ofnames[m]),\
                        '-outsum','{0}LPLsum_{1}.csv'.format(sfpath, ofnames[m]),\
                        '-clust2det','{0}LPLclust2det_{1}.csv'.format(sfpath, ofnames[m])])

Checking out argv[1] = -imgs.
Checking out argv[3] = -pairdets.
Checking out argv[5] = -lflist.
Checking out argv[7] = -outsum.
Checking out argv[9] = -clust2det.
input paired detection file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv
input cluster list file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_lflist
Maximum RMS in km: 100000
Maximum astrometric RMS: 1
Maximum fraction of points to be rejected: 0.5
Minimum number of observing nights: 3
Minimum number of unique detections: 6
output cluster file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLclust2det_pratchett_cart.csv
output rms file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLsum_pratchett_cart.csv
Reference MJD will be set to MJD-at-the-epoch from a previous orbit fit, if available.
Defaulting to simplex type = 1
Maximum fraction of points that can be rejected = 0.5, which is the default.
Maximum number of points that

In [27]:
## how many linkages?

a1_links = []
a2_links = []
a3_links = []

for n in range(len(ofnames)):

    if os.path.isfile('{0}LPLsum_{1}.csv'.format(sfpath, ofnames[n])):
        df_links = pd.read_csv('{0}LPLsum_{1}.csv'.format(sfpath, ofnames[n]))
        num_links = len(df_links)

        print(ofnames[n]+' - '+str(num_links))
    else:
        print('{0}LPLsum_{1}.csv does not exist'.format(sfpath, ofnames[n]))

pratchett_cart - 1
pratchett_ng_a1_-12.0 - 1
pratchett_ng_a2_-12.0 - 1
pratchett_ng_a3_-12.0 - 1
pratchett_ng_a1_-11.0 - 1
pratchett_ng_a2_-11.0 - 1
pratchett_ng_a3_-11.0 - 1
pratchett_ng_a1_-10.0 - 1
pratchett_ng_a2_-10.0 - 1
pratchett_ng_a3_-10.0 - 1
pratchett_ng_a1_-9.0 - 1
pratchett_ng_a2_-9.0 - 1
pratchett_ng_a3_-9.0 - 1
pratchett_ng_a1_-8.0 - 1
pratchett_ng_a2_-8.0 - 1
pratchett_ng_a3_-8.0 - 1
pratchett_ng_a1_-7.0 - 1
pratchett_ng_a2_-7.0 - 1
pratchett_ng_a3_-7.0 - 1
pratchett_ng_a1_-6.0 - 1
pratchett_ng_a2_-6.0 - 1
pratchett_ng_a3_-6.0 - 1
pratchett_ng_a1_-5.0 - 1
pratchett_ng_a2_-5.0 - 0
pratchett_ng_a3_-5.0 - 0
pratchett_ng_a1_-4.0 - 1
pratchett_ng_a2_-4.0 - 1
pratchett_ng_a3_-4.0 - 1
pratchett_ng_a1_-3.0 - 0
pratchett_ng_a3_-3.0 - 0
pratchett_ng_a1_-2.0 - 1
pratchett_ng_a3_-2.0 - 0
pratchett_ng_a1_-1.0 - 0
pratchett_ng_a2_-1.0 - 0
pratchett_ng_a3_-1.0 - 0
pratchett_ng_a1_0.0 - 0
pratchett_ng_a3_0.0 - 0
pratchett_ng_a1_1.0 - 0
pratchett_ng_a2_1.0 - 0
pratchett_ng_a3_1.0 - 0
pr